In [73]:
function temps(instance, liste_params)
    data_folder = string(join(split(@__DIR__, '\\')[1:end-1],'\\'),"\\data")
    wmd_file = joinpath(data_folder, join([instance, ".wmd"]))
    dat_file = joinpath(data_folder, join([instance, ".dat"]))
    
    global GRAPH = read_graph(wmd_file, dat_file)
    global VERTICES = 1:nv(GRAPH)
    global EDGES = [(e.src,e.dst) for e in edges(GRAPH)]
    global L = 3
    global WEIGHTS = Dict{Tuple{Int,Int},Float32}((e.src,e.dst) => get_prop(GRAPH, e, :weight) for e in edges(GRAPH))
    
    n_couples = nv(GRAPH)
    
    result_MIP = @timed solve_MIP()
    t_MIP = result_MIP.time
    
    t_BP = Array{Float32,1}(undef,length(liste_params))
    nb_noeuds_BP = Array{Float32,1}(undef,length(liste_params))
    
    for param in liste_params
        global BP_INIT_VERSION   = param[1]
        global BRANCHING_VERSION = param[2]
        global PROCESSING_RULE   = param[3]
        
        result_BP = @timed solve_BP()
        push!(t_BP, result_BP.time)
        push!(nb_noeuds_BP, result_BP.value[1])
    end
    
    return n_couples, t_MIP, t_BP, nb_noeuds_BP
end

temps (generic function with 3 methods)

In [ ]:
function compare(instances, liste_param)
    nb_instances = length(instances)
    nb_couples = Array{Float32,1}(undef, nb_instances)
    tps_MIP = Array{Float32,1}(undef, nb_instances)
    tps_BP = Array{Array{Float32,1},1}(undef, nb_instances)
    nb_noeuds = Array{Array{Float32,1},1}(undef, nb_instances)
    
    for i in 1:nb_instances
        nb_couples[i], tps_MIP[i], tps_BP[i], nb_noeuds[i] = temps(instances[i], liste_param)
    end
    
    tailles_instances = Set(nb_couples)
    mean_tps_MIP    = Array{Float32,1}(undef, length(tailles_instances))
    mean_tps_BP     = Array{Array{Float32,1},1}(undef,length(tailles_instances))
    mean_nb_noeuds  = Array{Array{Float32,1},1}(undef, length(tailles_instances))
    
    for t in 1:length(tailles_instances)
        indices = findall(nb_couples .== tailles_instances[t]) # indices des instances de dimension 'taille'
        push!(mean_tps_MIP, mean(tps_MIP[indices]))
        
        sum_tps = Array{Float32,1}(undef, length(liste_params))
        sum_nb_noeuds = Array{Float32,1}(undef, length(liste_params))
        for i in indices
            sum_tps = sum_tps .+ tps_BP[i]
            sum_nb_noeuds = sum_nb_noeuds .+ nb_noeuds[i]
        end
        mean_tps_BP[t] = sum_tps ./ length(indices)
        mean_nb_noeuds[t] = sum_nb_noeuds ./ length(indices)
    end
    
    # On retourne les indices ...
    mean_tps_BP_par_param = Array{Array{Float32,1},1}(undef, length(liste_params))
    mean_nb_noeuds_par_param = Array{Array{Float32,1},1}(undef, length(liste_params))
    for t in 1:length(tailles_instances)
        mean_tps_BP_par_param[t] = Array{Float32,1}(undef, length(tailles_instances))
        mean_nb_noeuds_par_param[t] = Array{Float32,1}(undef, length(tailles_instances))
        for p in 1:length(liste_params)
            mean_tps_BP_par_param[p][t] = mean_tps_BP[t][p]
            mean_nb_noeuds_par_param[p][t] = mean_nb_noeuds[t][p]
        end
    end
    
    
    
    plot(tailles_instances, mean_tps_MIP)
    for p in 1:length(liste_params)
        plot!(tailles_instances, mean_tps_BP_par_param[p])
    end
    
    plot(tailles_instances, mean_nb_noeuds_par_param[1])
    for p in 2:length(liste_params)
        plot!(tailles_instances, mean_nb_noeuds_par_param[p])
    end 

end